In [1]:
import pandas as pd
import json
import os
import numpy as np
import requests
import sqlalchemy as sql
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from dotenv import load_dotenv
from pathlib import Path
import datetime
%matplotlib inline

ModuleNotFoundError: No module named 'MCForecastTools'

In [ ]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

In [ ]:
# Set response URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"
ltc_url = "https://api.alternative.me/v2/ticker/Litecoin/?convert=USD"

In [ ]:
large_company_stocks=[
'AAPL',
'MSFT',
'AMZN',
'FB',
'TSLA',
'GOOGL']

medium_company_stocks=[
'GNRC',
'SEDG',
'PENN',
'CZR',
'TECH',
'CGNX']

small_company_stocks=[
'XSTLA',
'CLF',
'MSTR',
'NEO',
'DDD',
'OMCL']

In [ ]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response,indent=4,sort_keys=True))

In [ ]:
# Using the Python requests library, make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(eth_response,indent=4,sort_keys=True))

In [ ]:
# Using the Python requests library, make an API call to access the current price LTC
ltc_response = requests.get(ltc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(ltc_response,indent=4,sort_keys=True))

In [ ]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response['data']['1']['quotes']['USD']['price']

In [ ]:
# Navigate the ETH response object to access the current price of ETH
eth_price = eth_response['data']['1027']['quotes']['USD']['price']

In [ ]:
# Navigate the LTC response object to access the current price of LTC
ltc_price = ltc_response['data']['2']['quotes']['USD']['price']

In [ ]:
# remember to update date
today_date = str(datetime.datetime.now()).split()[0]
one_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=1*365)).split()[0]
three_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=3*365)).split()[0]
str(datetime.datetime.now()).split(" ")[0]
# Format current date as ISO format
today = pd.Timestamp(today_date, tz="America/New_York").isoformat()
today
# Format start and end dates as ISO format for one year period
start = pd.Timestamp(one_year_ago, tz="America/New_York").isoformat()
end = pd.Timestamp(today_date, tz="America/New_York").isoformat()
# Included for example
# # Get closing prices for FB and TWTR from the last year
# df_portfolio_year = alpaca.get_barset(
#     tickers,
#     timeframe,
#     start = start,
#     end = end
# ).df

In [ ]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key,alpaca_secret_key,api_version='v2')
display(type(alpaca_api_key)) # test to make sure api key imported successfully, str indicates import successful
display(type(alpaca_secret_key)) # test to make sure api key imported successfully, str indicates import successful

In [ ]:
# Set the tickers for both the bond and stock portion of the portfolio
tickers = ['SPY','QQQ','ARKK','AGG','O','VNQ']

# Set timeframe to 1D 
timeframe = '1D'

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
start_date = pd.Timestamp(today,tz='America/New_York').isoformat() # remember to update date
end_date = pd.Timestamp(today,tz='America/New_York').isoformat() # remember to update date

In [ ]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
prices_df = alpaca.get_barset(tickers,timeframe,start=start_date,end=end_date).df

# Review the first 5 rows of the Alpaca DataFrame
prices_df.head()

In [ ]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
agg_close_price = prices_df['AGG']['close'].item()

In [ ]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
spy_close_price = prices_df['SPY']['close'].item()

In [ ]:
# Access the closing price for QQQ from the Alpaca DataFrame
# Converting the value to a floating point number
qqq_close_price = prices_df['QQQ']['close'].item()

In [ ]:
# Access the closing price for ARKK from the Alpaca DataFrame
# Converting the value to a floating point number
arkk_close_price = prices_df['ARKK']['close'].item()

In [ ]:
# Access the closing price for O from the Alpaca DataFrame
# Converting the value to a floating point number
o_close_price = prices_df['O']['close'].item()

In [ ]:
# Access the closing price for VNQ from the Alpaca DataFrame
# Converting the value to a floating point number
vnq_close_price = prices_df['VNQ']['close'].item()
print(vnq_close_price)

In [ ]:
# Get user info
user_info = pd.read_csv('generic_user.csv')

In [ ]:
# Start to build portfolio options
user_info

In [ ]:
age = user_info['user_age']

In [ ]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user doesn't want crypto
portfolio_allocation_stocks_no_crypto = 100-age
portfolio_allocation_stocks_no_crypto

In [ ]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user does want crypto


In [ ]:
# Create basic portfolio models if user wants less customization
# Order is based [bonds, stocks, crypto]
conservative_portfolio = [.7,.3,0]
conservative_portfolio_crypto = [.7,.2,.1]
moderate_portfolio = [.5,.5,0]
moderate_portfolio_crypto = [.4,.4,.2]
aggressive_portfolio = [.3,.7,0]
aggressive_portfolio_crypto = [.2,.6,.2]
ultra_aggressive = [0,.7,.3]

In [ ]:
# GOAL: finish MVP by Tuesday, full by Thursday
# Build portfolios with each allocation
# Run monte carlos and graphs
# Finish CLI
# Test for bugs
# 